# Implementation of an bully algorithm without the Django framework
### This is a test, to know how it's build without the extra layers -> object based Python classes

In [1]:
import random

In [2]:
OWN_TRUCK_ID = 4
BROKEN_TRUCK_ID = 2
AMOUNT_OF_TRUCKS = 10

In [15]:
class Truck(object):
    def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broke: bool, convoy):
        self.truckId = truckId
        self.convoyPosition = convoyPosition
        self.convoyLeaderId = convoyLeaderId
        self.speed = speed
        self.broke = broke
        self.convoy = convoy
    
    def alive(self):
        return not self.broke
    
    def toggleBroke(self):
        self.broke = not self.broke
    
    def getTruckId(self):
        return self.truckId
    
    def setConvoyPosition(self, convoyPosition):
        self.convoyPosition = convoyPosition
        
    def getConvoyPosition(self):
        return self.convoyPosition
    
    def setConvoyLeaderId(self, convoyLeaderId):
        self.convoyLeaderId = convoyLeaderId
        
    def getConvoyLeaderId(self):
        return self.convoyLeaderId
    
    def setSpeed(self, speed):
        self.speed = speed
        
    def getSpeed(self):
        return self.speed
    
    def startPoll(self):
        #answer = {'pos': ('alive', 'truckId')}
        oldConvoyPosition = self.convoyPosition
        
        trucks = self.convoy.getTrucks()
        trucksAlive = {}
        for convoyPosition, truck in trucks.items():
            if convoyPosition <= self.convoyPosition:
                trucksAlive[convoyPosition] = (truck.alive(), truck.getTruckId())
        newConvoyPosition = self.__bully__(trucksAlive)
        self.convoyPosition = newConvoyPosition
        return oldConvoyPosition - newConvoyPosition
    
    def __bully__(self, trucksAlive):
        newPositions = []
        for position, (alive, truckId) in trucksAlive.items():
            if alive:
                newPositions.append(truckId)
        return newPositions.index(self.truckId)
        #self.truckId = newPositions.index(self.truckId)

In [16]:
class Convoy(object):
    def __init__(self, amountOfTrucks: int):
        self.truckIds = []
        self.trucks = {}
        self.__createTrucks__(amountOfTrucks)
    
    def __createTrucks__(self, amountOfTrucks: int):
        convoyLeaderId = 0
        speed = 35.0
        broke = False
        for convoyPosition in range(amountOfTrucks):
            truckId = self.__randomTruckId__()
            newTruck = Truck(truckId, convoyPosition, convoyLeaderId, speed, broke, self)
            self.trucks[convoyPosition] = newTruck
    
    def __randomTruckId__(self):
        success = False
        while not success:
            newId = random.randint(1, 1000)
            if not newId in self.truckIds:
                return newId
    
    def getTruckAt(self, convoyPosition: int): # Das ist dann nachher die Netzwerkadresse des Trucks
        return self.trucks[convoyPosition]
    
    def getTrucks(self):
        return self.trucks
    
    def de_activateRandomTruck(self):
        amount = self.getAmountOfTrucks()
        rand = random.randint(0, amount-1)
        self.trucks[rand].toggleBroke()
    
    def getAmountOfTrucks(self):
        return len(self.trucks)
    
    def geta(self):
        return self.trucks[9].startPoll()

In [17]:
convoy = Convoy(AMOUNT_OF_TRUCKS)

In [18]:
trucks = convoy.getTrucks()

for key, val in trucks.items():
    print(f'Nummer: {key}\nTruck: {val.getTruckId()}\n\n')

Nummer: 0
Truck: 575


Nummer: 1
Truck: 577


Nummer: 2
Truck: 210


Nummer: 3
Truck: 680


Nummer: 4
Truck: 7


Nummer: 5
Truck: 649


Nummer: 6
Truck: 314


Nummer: 7
Truck: 767


Nummer: 8
Truck: 969


Nummer: 9
Truck: 990




In [19]:
convoy.de_activateRandomTruck()
convoy.de_activateRandomTruck()
convoy.de_activateRandomTruck()

a = convoy.geta()
print(a)

3


In [21]:
trucks = convoy.getTrucks()

for key, val in trucks.items():
    print(f'Nummer: {key}\nTruckPosition: {val.getConvoyPosition()}\nTruck: {val.getTruckId()}\nAlive: {val.alive()}\n\n')

Nummer: 0
TruckPosition: 0
Truck: 575
Alive: False


Nummer: 1
TruckPosition: 1
Truck: 577
Alive: False


Nummer: 2
TruckPosition: 2
Truck: 210
Alive: True


Nummer: 3
TruckPosition: 3
Truck: 680
Alive: False


Nummer: 4
TruckPosition: 4
Truck: 7
Alive: True


Nummer: 5
TruckPosition: 5
Truck: 649
Alive: True


Nummer: 6
TruckPosition: 6
Truck: 314
Alive: True


Nummer: 7
TruckPosition: 7
Truck: 767
Alive: True


Nummer: 8
TruckPosition: 8
Truck: 969
Alive: True


Nummer: 9
TruckPosition: 6
Truck: 990
Alive: True




In [14]:
# rekursives arbeiten mit dem poll, d.h. die poll wird von unten nach oben angestossen und dann rekursiv ueber
# die element ids ausgefuehrt, nachdem die rueckmeldung gegeben wurde

# NOTES

- TruckID wird durch Convoy Microservice zugewiesen
- Convoy Microservice muss eine Liste des Convoys haben
- Indizes der Liste sind die ConvoyPosition
- Reihenfolge der TruckIds in der Convoy Liste ist scheifegal